# Generate SSH Keys on [Commjhub](https://commjhub.asc.upenn.edu/) for GitHub 🚀🔑⚡

Etienne P Jacquot - [epj@asc.upenn.edu](mailto:epj@asc.upenn.edu)

- Last edited: 08/28/2023

________

## Getting Started

Following official instructions [here](https://docs.github.com/en/github/authenticating-to-github/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent).

> We do *NOT* want to have undergraduate students entering passwords in plain text -- Configuring SSH keys is a must!!!

_________________

### 1. Create your SSH key & add to your SSH-Agent:

- Default directory is for `$HOME/.ssh/id_ed25519` by including `-f` flag

- We default to **NO** password by including `-N ""` flag & empty phrase

- We include the `<<< y` to overwrite an existing key!

- the `&& \` joins multiple bash commands and wraps text (*i.e. for formatting a one liner*)


In [1]:
%%bash
ssh-keygen -t ed25519 -C $USER"@commjhub.asc.upenn.edu" -f $HOME/.ssh/id_ed25519 -N "" <<< y && \
eval "$(ssh-agent -s)" && \
ssh-add $HOME/.ssh/id_ed25519

Generating public/private ed25519 key pair.
/Commjhub/jupyterhub/home/atnjqt/.ssh/id_ed25519 already exists.
Overwrite (y/n)? Your identification has been saved in /Commjhub/jupyterhub/home/atnjqt/.ssh/id_ed25519
Your public key has been saved in /Commjhub/jupyterhub/home/atnjqt/.ssh/id_ed25519.pub
The key fingerprint is:
SHA256:MwAaHgG754s3egdHYb3DW35CJeBKM1RczTO3vD7bp+c atnjqt@commjhub.asc.upenn.edu
The key's randomart image is:
+--[ED25519 256]--+
|..+.o+oo.o       |
| o =ooo . * .    |
|. o.+oo. o = .   |
| . ..++.o   o    |
|. ...  *S    .   |
| o. . . oo. .    |
|  .o     o .     |
| .+..       o.  o|
|o+.o        .oo=E|
+----[SHA256]-----+
Agent pid 2066110


Identity added: /Commjhub/jupyterhub/home/atnjqt/.ssh/id_ed25519 (atnjqt@commjhub.asc.upenn.edu)


_______
### 2. Copy Public SSH key from Commjhub


Copy the full result of `.ssh/id_ed25519.pub` to your clipboard

> **Remember to never share your private key! If compromised, you need to simple regenerate.**
> - As in, do not share `.ssh/id_ed25519`! That is like sharing a blueprint to replicate your house key 🔑

In [3]:
%%bash
cat $HOME/.ssh/id_ed25519.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAINCPydxnLFoLG+8nKS1+m1pGbCsEWeQ1411kvjZRTjL8 atnjqt@commjhub.asc.upenn.edu


____________
### 3. Upload Public SSH key to Github

Navigate to [https://github.com/settings/keys](https://github.com/settings/keys) and upload your public SSH key (more info [here](https://docs.github.com/en/github/authenticating-to-github/adding-a-new-ssh-key-to-your-github-account)).

> If your GitHub account has MFA enabled, you may need to confirm two factor authentication during the key upload

____________

## 4. Testing your SSH Access to Github.com

Once your SSH public key is uploaded to github, proceed to securely test your connection (more info [here](https://docs.github.com/en/github/authenticating-to-github/testing-your-ssh-connection)).


### Retrieve SSH host information & RSA fingerprint over the internet

- *Manually navigate to obtain the **Official RSA public key fingerprint for Github.com** [here](https://docs.github.com/en/github/authenticating-to-github/githubs-ssh-key-fingerprints)*
    - You must set this as your `rsa_Ed25519=SHA256:...` value in the cell below!
   



- We then scan with `ssh-keyscan` and write response to local file [./github_host_info](./github_host_info). We then generate the fingerprint hash for our scanned host information and write to another local file [./fingerprint_rsa](./fingerprint_rsa). 


> *This allows us to compare the trusted host identifier with the actual host (untrusted) identifier we get over public internet, to then safely add to our known_hosts file... 
This example is based on a helpful serverfault question [here](https://serverfault.com/questions/856194/securely-add-a-host-e-g-github-to-the-ssh-known-hosts-file).*


In [4]:
%%bash
rm -rf $HOME/.ssh/known_hosts
#export rsa_fp='SHA256:nThbg6kXUpJWGl7E1IGOCspRomTxdCARLviKw6E5SY8' # <-- ADD LEGIT VALUE HERE
export rsa_Ed25519='SHA256:+DiY3wvvV6TuJJhbpZisF/zLDA0zPMSvHdkr4UvCOqU' # <-- ADD LEGIT VALUE HERE

ssh-keyscan -t ed25519 github.com | tee ./github_host_info | ssh-keygen -lf - >> ./fingerprint_ed25519
set -e # <-- Bash will break on error (i.e. if python assertion fails)
python -c "assert('$rsa_Ed25519' == open('./fingerprint_ed25519', 'r').read().split()[1])"
cat ./github_host_info >> $HOME/.ssh/known_hosts

# github.com:22 SSH-2.0-babeld-22243ffb


#### *If the above assertion fails...*

Then the value was *not* added to your known hosts file... Try to resolve with:

- confirm you set the correct `rsa_pubkey_fingerprint` from [here](https://docs.github.com/en/github/authenticating-to-github/githubs-ssh-key-fingerprints)
- confirm there is no man in the middle attack for github.com on your local network (you have bigger problems if that is the case!)...
________________

## 5. Finally, Confirm SSH Connectivity to Github

To confirm your access:
- we avoid `-StrictHostKeyChecking no` flag which is obviously insecure

In [5]:
!ssh -T git@github.com 

Hi atnjqt! You've successfully authenticated, but GitHub does not provide shell access.


#### *If you get a failed reponse, read carefully!*

This should return your username, confirming you did everything correctly in setting up SSH for github!
```
Warning: Permanently added the RSA host key for IP address '140.82.113.4' to the list of known hosts.
Hi atnjqt! You've successfully authenticated, but GitHub does not provide shell access.
```

#### *If you do actually get an error message*

For example:

```bash
git@github.com: Permission denied (publickey).
```
- you should restart this notebook & start fresh, including the upload of public ssh key to github
___________

### Questions?

Please reach out to [etienne.jacquot@asc.upenn.edu](mailto:etienne.jacquot@asc.upenn.edu) or on the github repo [https://github.com/atnjqt/ssh-keygen-jhub](https://github.com/atnjqt/ssh-keygen-jhub)